In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers datasets torch pillow bert_score tqdm nltk pandas matplotlib evaluate rouge_score py-rouge sacrebleu ftfy regex
!pip install open-clip-torch  # Correct package name with hyphens

import os
import random
import torch
import numpy as np
import pandas as pd
import nltk
import evaluate
import open_clip
from PIL import Image
from collections import Counter
from datasets import load_dataset
from tqdm import tqdm
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import BartTokenizer, BartForConditionalGeneration

# Import the rouge_scorer package for ROUGE-L calculation
try:
    from rouge_score import rouge_scorer
except ImportError:
    # Install if not available
    !pip install rouge-score
    from rouge_score import rouge_scorer

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.6 MB/s eta 0:00:00
   ━━

2025-05-11 06:11:08.022704: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746943868.241534      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746943868.305256      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Step 1: Verify directory structure
print("Input directory:", os.listdir('/kaggle/input/'))
print("vqa-dataset contents:", os.listdir('/kaggle/input/dataset'))
print("small folder contents:", os.listdir('/kaggle/input/imagedataset/small'))

# Step 2: Load the full dataset from CSV
csv_path = '/kaggle/input/dataset/Complete_vqa(in).csv'
full_dataset = load_dataset("csv", data_files=csv_path)

# Step 3: Update image paths to absolute paths
base_image_path = '/kaggle/input/imagedataset/small'
full_dataset = full_dataset.map(lambda example: {'image_path': os.path.join(base_image_path, example['image_path'])})

# Step 4: Load the CLIP model
print("Loading CLIP model and processor...")
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
model = model.to(device)

# Load BART model for BARTScore calculation
print("Loading BART model for BARTScore...")
from transformers import BartTokenizer, BartForConditionalGeneration
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
bart_model.eval()
bart_model = bart_model.to(device)

# Initialize evaluation metrics
bertscore = evaluate.load('bertscore')

# Define answer candidates (common VQA answers)
common_answers = [
    "yes", "no", "2", "1", "white", "red", "blue", "black", "3", "green",
    "yellow", "4", "brown", "orange", "0", "5", "gray", "grey", "purple", "pink",
    "6", "7", "8", "9", "10", "standing", "sitting", "right", "left", "wood",
    "metal", "plastic", "glass", "paper", "water", "food", "man", "woman", "boy",
    "girl", "dog", "cat", "table", "chair", "car", "phone", "book", "computer",
    "tv", "bed", "outside", "inside", "day", "night", "morning", "evening", "afternoon",
    "kitchen", "bathroom", "bedroom", "living room", "large", "small", "medium",
    "round", "square", "rectangular", "triangle", "circle", "diamond", "heart", "star",
    "up", "down", "on", "under", "near", "far", "tall", "short", "long", "wide",
    "narrow", "big", "little", "high", "low", "hot", "cold", "warm", "cool", "old",
    "new", "young", "open", "closed"
]

# Step 5: Define preprocessing function
def preprocess_example(example):
    image_path = example['image_path']
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")
    
    # Load and preprocess the image for CLIP
    image = preprocess(Image.open(image_path).convert("RGB")).unsqueeze(0).to(device)
    
    # Prepare the question
    question = example['question']
    
    return {'image': image, 'question': question}

# Step 6: Define VQA prediction function using CLIP
def get_prediction(example):
    try:
        processed = preprocess_example(example)
        image = processed['image']
        question = processed['question']
        
        # Encode image
        with torch.no_grad():
            image_features = model.encode_image(image)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            
            # Prepare text prompts (question + possible answer)
            answer_prompts = [f"Question: {question} Answer: {ans}" for ans in common_answers]
            
            # Use open_clip's tokenizer
            tokenizer = open_clip.get_tokenizer('ViT-B-32')
            text = tokenizer(answer_prompts).to(device)
            
            # Encode text prompts
            text_features = model.encode_text(text)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)
            
            # Calculate similarity scores
            similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            
            # Get the most likely answer
            values, indices = similarity[0].topk(1)
            predicted_answer = common_answers[indices[0].item()]
            
        return predicted_answer
    except Exception as e:
        print(f"Error in prediction: {e}")
        return ""

# Step 7: Create sampling functions for different dataset sizes
def create_sampled_dataset(dataset, percentage):
    dataset_size = len(dataset['train'])
    sample_size = int(dataset_size * percentage)
    
    # Get random indices without replacement
    indices = random.sample(range(dataset_size), sample_size)
    
    # Create sampled dataset
    sampled_data = dataset['train'].select(indices)
    return sampled_data

# Step 8: Define evaluation metrics
def calculate_metrics(predictions, ground_truths):
    metrics = {}
    
    # Ensure predictions and ground_truths are valid
    valid_pairs = [(p, g) for p, g in zip(predictions, ground_truths) 
                    if isinstance(p, str) and isinstance(g, str)]
    
    if not valid_pairs:
        return {
            'accuracy': 0.0,
            'f1_score': 0.0,
            'bleu_score': 0.0,
            'bert_score_p': 0.0,
            'bert_score_r': 0.0,
            'bert_score_f1': 0.0,
            'bart_score': 0.0,
            'rouge_l': 0.0
        }
    
    valid_predictions, valid_ground_truths = zip(*valid_pairs)
    
    # Accuracy calculation
    correct = sum(1 for p, g in zip(valid_predictions, valid_ground_truths) 
                 if p.lower() == g.lower())
    metrics['accuracy'] = correct / len(valid_pairs) if valid_pairs else 0.0
    
    # F1 Score calculation (token level)
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    for pred, truth in valid_pairs:
        pred_tokens = set(pred.lower().split())
        truth_tokens = set(truth.lower().split())
        
        true_positives += len(pred_tokens.intersection(truth_tokens))
        false_positives += len(pred_tokens - truth_tokens)
        false_negatives += len(truth_tokens - pred_tokens)
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    metrics['f1_score'] = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # BLEU Score
    smoothie = SmoothingFunction().method1
    bleu_scores = []
    
    for pred, truth in valid_pairs:
        pred_tokens = nltk.word_tokenize(pred.lower())
        truth_tokens = [nltk.word_tokenize(truth.lower())]
        try:
            bleu = sentence_bleu(truth_tokens, pred_tokens, smoothing_function=smoothie)
            bleu_scores.append(bleu)
        except Exception as e:
            print(f"BLEU calculation error: {e}")
            bleu_scores.append(0)
    
    metrics['bleu_score'] = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
    
    # BERTScore using evaluate library
    try:
        bert_results = bertscore.compute(
            predictions=valid_predictions, 
            references=valid_ground_truths, 
            lang="en",
            model_type="microsoft/deberta-xlarge-mnli"
        )
        metrics['bert_score_p'] = sum(bert_results['precision']) / len(bert_results['precision'])
        metrics['bert_score_r'] = sum(bert_results['recall']) / len(bert_results['recall'])
        metrics['bert_score_f1'] = sum(bert_results['f1']) / len(bert_results['f1'])
    except Exception as e:
        print(f"BERTScore calculation error with evaluate library: {e}")
        # Fallback to bert_score package
        try:
            # Using smaller batches to avoid memory issues
            batch_size = 32
            all_bert_p = []
            all_bert_r = []
            all_bert_f1 = []
            
            for i in range(0, len(valid_predictions), batch_size):
                batch_preds = valid_predictions[i:i+batch_size]
                batch_refs = valid_ground_truths[i:i+batch_size]
                
                P, R, F1 = bert_score_calc(batch_preds, batch_refs, lang="en", verbose=False)
                all_bert_p.extend(P.tolist())
                all_bert_r.extend(R.tolist())
                all_bert_f1.extend(F1.tolist())
            
            metrics['bert_score_p'] = sum(all_bert_p) / len(all_bert_p) if all_bert_p else 0
            metrics['bert_score_r'] = sum(all_bert_r) / len(all_bert_r) if all_bert_r else 0
            metrics['bert_score_f1'] = sum(all_bert_f1) / len(all_bert_f1) if all_bert_f1 else 0
        except Exception as e:
            print(f"BERTScore fallback calculation error: {e}")
            metrics['bert_score_p'] = 0
            metrics['bert_score_r'] = 0
            metrics['bert_score_f1'] = 0
    
    # BARTScore calculation
    bart_scores = []
    try:
        with torch.no_grad():
            for pred, ref in tqdm(list(zip(valid_predictions, valid_ground_truths)), desc="Calculating BARTScore", leave=False):
                # Skip empty strings
                if not pred.strip() or not ref.strip():
                    continue
                
                # Encode the prediction and reference
                inputs = bart_tokenizer(ref, return_tensors="pt", max_length=1024, truncation=True).to(device)
                with torch.no_grad():
                    pred_encoded = bart_tokenizer(pred, return_tensors="pt", max_length=1024, truncation=True).to(device)
                    ref_ids = inputs["input_ids"]
                    
                    # Calculate log likelihood
                    outputs = bart_model(
                        input_ids=pred_encoded["input_ids"],
                        attention_mask=pred_encoded["attention_mask"],
                        labels=ref_ids
                    )
                    neg_log_likelihood = outputs.loss.item()
                    # Convert negative log likelihood to score (higher is better)
                    bart_score = -neg_log_likelihood
                    bart_scores.append(bart_score)
        
        metrics['bart_score'] = sum(bart_scores) / len(bart_scores) if bart_scores else 0
    except Exception as e:
        print(f"BARTScore calculation error: {e}")
        metrics['bart_score'] = 0
    
    # ROUGE-L Score
    try:
        scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        rouge_scores = []
        
        for pred, truth in valid_pairs:
            score = scorer.score(truth, pred)
            rouge_scores.append(score['rougeL'].fmeasure)
        
        metrics['rouge_l'] = sum(rouge_scores) / len(rouge_scores) if rouge_scores else 0
    except Exception as e:
        print(f"ROUGE-L calculation error: {e}")
        metrics['rouge_l'] = 0
    
    return metrics

# Step 9: Define evaluation function with progress bar
def evaluate_dataset(dataset_sample, name):
    print(f"\nEvaluating {name} dataset ({len(dataset_sample)} samples)...")
    predictions = []
    ground_truths = []
    
    # Use tqdm for progress bar
    for example in tqdm(dataset_sample, desc=f"Processing {name} dataset"):
        try:
            predicted_answer = get_prediction(example)
            ground_truth = example['answer']
            
            # Skip if ground_truth or predicted_answer is invalid
            if (ground_truth is None or not isinstance(ground_truth, str) or
                predicted_answer is None or not isinstance(predicted_answer, str)):
                continue
            
            predictions.append(predicted_answer)
            ground_truths.append(ground_truth)
        
        except FileNotFoundError as e:
            print(f"FileNotFoundError: {e}")
            continue
        except Exception as e:
            print(f"Error processing example: {e}")
            continue
    
    # Calculate metrics
    metrics = calculate_metrics(predictions, ground_truths)
    
    # Print metrics
    print(f"\n{name} Evaluation Results:")
    print(f"Number of samples evaluated: {len(predictions)}")
    print(f"Accuracy: {metrics['accuracy'] * 100:.2f}%")
    print(f"F1 Score: {metrics['f1_score'] * 100:.2f}%")
    print(f"BLEU Score: {metrics['bleu_score'] * 100:.2f}%")
    print(f"BERTScore Precision: {metrics['bert_score_p'] * 100:.2f}%")
    print(f"BERTScore Recall: {metrics['bert_score_r'] * 100:.2f}%")
    print(f"BERTScore F1: {metrics['bert_score_f1'] * 100:.2f}%")
    print(f"BARTScore: {metrics['bart_score']:.4f}")
    print(f"ROUGE-L: {metrics['rouge_l'] * 100:.2f}%")
    
    return metrics

# Step 10: Create different sized datasets
print(f"\nTotal dataset size: {len(full_dataset['train'])} samples")

# Create different sized datasets
full_sample = full_dataset['train']
half_sample = create_sampled_dataset(full_dataset, 0.5)
quarter_sample = create_sampled_dataset(full_dataset, 0.25)

print(f"Full dataset: {len(full_sample)} samples")
print(f"50% dataset: {len(half_sample)} samples")
print(f"25% dataset: {len(quarter_sample)} samples")

# Step 11: Evaluate and collect results
results = {}

# Evaluate quarter dataset
results['25% Dataset'] = evaluate_dataset(quarter_sample, "25% Dataset")

# Evaluate half dataset
results['50% Dataset'] = evaluate_dataset(half_sample, "50% Dataset")

# Evaluate full dataset
results['Full Dataset'] = evaluate_dataset(full_sample, "Full Dataset")

# Step 12: Create comparison charts
def plot_metrics_comparison(results):
    metrics_to_plot = ['accuracy', 'f1_score', 'bleu_score', 'bert_score_f1', 'bart_score', 'rouge_l']
    display_names = {
        'accuracy': 'Accuracy',
        'f1_score': 'F1 Score',
        'bleu_score': 'BLEU Score',
        'bert_score_f1': 'BERTScore F1',
        'bart_score': 'BARTScore',
        'rouge_l': 'ROUGE-L'
    }
    
    # Create figure with subplots - 2 rows of 3 metrics each
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Plot each metric
    axes = axes.flatten()
    
    for i, metric in enumerate(metrics_to_plot):
        dataset_names = list(results.keys())
        
        # Handle BARTScore differently as it's not a percentage
        if metric == 'bart_score':
            values = [results[dataset][metric] for dataset in dataset_names]
            bars = axes[i].bar(dataset_names, values)
            axes[i].set_title(display_names[metric])
            axes[i].set_ylabel('Score')
            
            # Add value labels on bars
            for bar in bars:
                height = bar.get_height()
                axes[i].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                             f'{height:.4f}', ha='center', va='bottom')
        else:
            values = [results[dataset][metric] * 100 for dataset in dataset_names]
            bars = axes[i].bar(dataset_names, values)
            axes[i].set_title(display_names[metric])
            axes[i].set_ylabel('Score (%)')
            axes[i].set_ylim(0, 100)
            
            # Add value labels on bars
            for bar in bars:
                height = bar.get_height()
                axes[i].text(bar.get_x() + bar.get_width()/2., height + 1,
                             f'{height:.2f}%', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig('clip_metrics_comparison.png')
    plt.close()
    
    # Create dataset size comparison
    dataset_sizes = {
        '25% Dataset': len(quarter_sample),
        '50% Dataset': len(half_sample),
        'Full Dataset': len(full_sample)
    }
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(dataset_sizes.keys(), dataset_sizes.values())
    plt.title('Dataset Size Comparison')
    plt.ylabel('Number of Samples')
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                 f'{height}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig('dataset_size_comparison.png')
    plt.close()

# Generate comparison plots
plot_metrics_comparison(results)

# Save results to CSV
results_df = pd.DataFrame({
    'Dataset': list(results.keys()),
    'Size': [len(quarter_sample), len(half_sample), len(full_sample)],
    'Accuracy (%)': [results[ds]['accuracy'] * 100 for ds in results],
    'F1 Score (%)': [results[ds]['f1_score'] * 100 for ds in results],
    'BLEU Score (%)': [results[ds]['bleu_score'] * 100 for ds in results],
    'BERTScore P (%)': [results[ds]['bert_score_p'] * 100 for ds in results],
    'BERTScore R (%)': [results[ds]['bert_score_r'] * 100 for ds in results],
    'BERTScore F1 (%)': [results[ds]['bert_score_f1'] * 100 for ds in results],
    'BARTScore': [results[ds]['bart_score'] for ds in results],
    'ROUGE-L (%)': [results[ds]['rouge_l'] * 100 for ds in results]
})

results_df.to_csv('clip_evaluation_results.csv', index=False)
print("\nResults saved to clip_evaluation_results.csv")
print("\nEvaluation complete!")

Input directory: ['imagedataset', 'dataset']
vqa-dataset contents: ['Complete_vqa(in).csv']
small folder contents: ['d8', '0d', '47', '17', '81', 'c5', '19', 'e2', '8f', 'a2', '22', 'cf', 'b8', '35', '92', 'b2', '50', 'c7', '23', '5b', '87', '07', 'd5', '10', 'd0', 'cd', '61', '1b', 'bf', '2e', '36', '05', '20', '06', '0f', '45', '60', '27', '6d', '64', '41', 'c8', '9e', 'da', 'ff', '89', '39', '0c', '4d', '02', '32', '98', '25', '42', 'b1', 'e5', 'b4', '52', 'f0', 'ac', 'd9', 'e0', '75', '0a', '8b', '2c', '38', '7d', '12', 'dc', 'ef', '94', '55', 'a0', 'e1', 'e7', '04', 'be', 'f1', '7e', 'ea', '49', 'fc', 'e3', 'e4', 'b9', '6e', '31', 'f9', '62', 'b6', '53', '1a', 'd2', '70', '34', '18', '4b', 'db', '79', '85', 'c2', '88', '65', '1e', '67', 'ec', 'ab', 'a3', 'a7', '78', '0e', 'd6', '28', 'f4', '66', 'cb', 'a5', 'images.csv', 'a8', 'bc', 'e6', '56', '72', '6c', '16', '7b', 'bb', 'af', '9f', 'bd', 'ba', '13', '99', '1d', '4e', '3e', '7f', '26', 'b5', 'fb', 'd4', '4c', '74', 'b0', 'c6', 

Processing 25% Dataset dataset:   0%|          | 0/11370 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

Calculating BARTScore:   0%|          | 0/11360 [00:00<?, ?it/s]


25% Dataset Evaluation Results:
Number of samples evaluated: 11360
Accuracy: 9.92%
F1 Score: 9.89%
BLEU Score: 1.76%
BERTScore Precision: 58.12%
BERTScore Recall: 57.61%
BERTScore F1: 57.72%
BARTScore: -6.7001
ROUGE-L: 10.04%

Evaluating 50% Dataset dataset (22741 samples)...


Processing 50% Dataset dataset:   0%|          | 0/22741 [00:00<?, ?it/s]

In [8]:
# Step 1: Verify directory structure
print("Input directory:", os.listdir('/kaggle/input/'))
print("vqa-dataset contents:", os.listdir('/kaggle/input/dataset'))
print("small folder contents:", os.listdir('/kaggle/input/imagedataset/small'))

# Step 2: Load the full dataset from CSV
csv_path = '/kaggle/input/dataset/Complete_vqa(in).csv'
full_dataset = load_dataset("csv", data_files=csv_path)

# Step 3: Update image paths to absolute paths
base_image_path = '/kaggle/input/imagedataset/small'
full_dataset = full_dataset.map(lambda example: {'image_path': os.path.join(base_image_path, example['image_path'])})

# Step 4: Load the CLIP model
print("Loading CLIP model and processor...")
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
model = model.to(device)

# Load BART model for BARTScore calculation
print("Loading BART model for BARTScore...")
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
bart_model.eval()
bart_model = bart_model.to(device)

# Initialize evaluation metrics
bertscore = evaluate.load('bertscore')

# Step 5: Extract all unique answers from the dataset (optimized for one-word answers)
def extract_unique_answers(dataset):
    # Get all answers and convert to lowercase for consistency
    all_answers = [example['answer'].lower().strip() for example in dataset 
                  if isinstance(example['answer'], str) and example['answer'].strip()]
    
    # Count answer frequencies
    answer_counts = Counter(all_answers)
    
    # For one-word answers, we can be more inclusive with frequency filtering
    # but still filter very rare answers to avoid overfitting
    min_count = 5  # Adjust based on dataset size
    filtered_answers = [ans for ans, count in answer_counts.items() if count >= min_count]
    
    # Explicitly filter to ensure we only have single-word answers
    # This helps with matching efficiency and removes potential multi-word outliers
    single_word_answers = [ans for ans in filtered_answers if len(ans.split()) == 1]
    
    # Sort by frequency (prioritizing common answers)
    single_word_answers.sort(key=lambda x: answer_counts[x], reverse=True)
    
    print(f"Extracted {len(single_word_answers)} unique single-word answers from dataset")
    print(f"Original dataset had {len(answer_counts)} distinct answers")
    print(f"After frequency filtering ({min_count}+ occurrences): {len(filtered_answers)} answers")
    print(f"After ensuring single-word only: {len(single_word_answers)} answers")
    
    return single_word_answers

# Extract unique answers
unique_answers = extract_unique_answers(full_dataset['train'])

# Step 6: Define preprocessing function
def preprocess_example(example):
    image_path = example['image_path']
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")
    
    # Load and preprocess the image for CLIP
    image = preprocess(Image.open(image_path).convert("RGB")).unsqueeze(0).to(device)
    
    # Prepare the question
    question = example['question']
    
    return {'image': image, 'question': question}

# Step 7: Define VQA prediction function using CLIP (optimized for single-word answers)
def get_prediction(example, unique_answers, batch_size=512):
    try:
        processed = preprocess_example(example)
        image = processed['image']
        question = processed['question']
        
        # Encode image once
        with torch.no_grad():
            image_features = model.encode_image(image)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            
            # For single-word answers, we can use larger batch sizes safely
            # We can also use more specific prompt templates for better accuracy
            best_score = -float('inf')
            best_answer = ""
            
            for i in range(0, len(unique_answers), batch_size):
                batch_answers = unique_answers[i:i+batch_size]
                
                # Create more specific prompts for single-word answers
                # This template helps CLIP distinguish between similar concepts better
                answer_prompts = [f"The answer to the question '{question}' is {ans}." for ans in batch_answers]
                
                # Use open_clip's tokenizer
                tokenizer = open_clip.get_tokenizer('ViT-B-32')
                text = tokenizer(answer_prompts).to(device)
                
                # Encode text prompts
                text_features = model.encode_text(text)
                text_features = text_features / text_features.norm(dim=-1, keepdim=True)
                
                # Calculate similarity scores
                similarity = (100.0 * image_features @ text_features.T)
                
                # Update best answer if found
                batch_best_score, batch_best_idx = similarity[0].max(dim=0)
                if batch_best_score > best_score:
                    best_score = batch_best_score
                    best_answer = batch_answers[batch_best_idx.item()]
            
        return best_answer
    except Exception as e:
        print(f"Error in prediction: {e}")
        return ""

# Step 8: Create sampling functions for different dataset sizes
def create_sampled_dataset(dataset, percentage):
    dataset_size = len(dataset['train'])
    sample_size = int(dataset_size * percentage)
    
    # Get random indices without replacement
    indices = random.sample(range(dataset_size), sample_size)
    
    # Create sampled dataset
    sampled_data = dataset['train'].select(indices)
    return sampled_data

# Import necessary NLTK components
try:
    from nltk.metrics.distance import edit_distance
except ImportError:
    nltk.download('punkt')
    from nltk.metrics.distance import edit_distance

# Step 9: Define evaluation metrics (optimized for single-word answers)
def calculate_metrics(predictions, ground_truths):
    metrics = {}
    
    # Ensure predictions and ground_truths are valid
    valid_pairs = [(p, g) for p, g in zip(predictions, ground_truths) 
                    if isinstance(p, str) and isinstance(g, str)]
    
    if not valid_pairs:
        return {
            'accuracy': 0.0,
            'f1_score': 0.0,
            'bleu_score': 0.0,
            'bert_score_p': 0.0,
            'bert_score_r': 0.0,
            'bert_score_f1': 0.0,
            'bart_score': 0.0,
            'rouge_l': 0.0
        }
    
    valid_predictions, valid_ground_truths = zip(*valid_pairs)
    
    # For single-word answers, exact match accuracy is the most important metric
    # Normalize everything to lowercase single words for fair comparison
    norm_predictions = [p.lower().strip() for p in valid_predictions]
    norm_ground_truths = [g.lower().strip() for g in valid_ground_truths]
    
    # Exact match accuracy
    correct = sum(1 for p, g in zip(norm_predictions, norm_ground_truths) if p == g)
    metrics['accuracy'] = correct / len(valid_pairs) if valid_pairs else 0.0
    
    # Also calculate "close match" accuracy (for words that might have slight spelling variations)
    # This is particularly useful for single-word answers
    close_matches = 0
    for p, g in zip(norm_predictions, norm_ground_truths):
        # If exact match or Levenshtein distance <= 1 (handles minor typos)
        if p == g or (len(p) > 2 and len(g) > 2 and nltk.edit_distance(p, g) <= 1):
            close_matches += 1
    
    metrics['close_match_accuracy'] = close_matches / len(valid_pairs) if valid_pairs else 0.0
    
    # F1 Score calculation (token level)
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    for pred, truth in valid_pairs:
        pred_tokens = set(pred.lower().split())
        truth_tokens = set(truth.lower().split())
        
        true_positives += len(pred_tokens.intersection(truth_tokens))
        false_positives += len(pred_tokens - truth_tokens)
        false_negatives += len(truth_tokens - pred_tokens)
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    metrics['f1_score'] = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # BLEU Score
    smoothie = SmoothingFunction().method1
    bleu_scores = []
    
    for pred, truth in valid_pairs:
        pred_tokens = nltk.word_tokenize(pred.lower())
        truth_tokens = [nltk.word_tokenize(truth.lower())]
        try:
            bleu = sentence_bleu(truth_tokens, pred_tokens, smoothing_function=smoothie)
            bleu_scores.append(bleu)
        except Exception as e:
            print(f"BLEU calculation error: {e}")
            bleu_scores.append(0)
    
    metrics['bleu_score'] = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
    
    # BERTScore using evaluate library
    try:
        bert_results = bertscore.compute(
            predictions=valid_predictions, 
            references=valid_ground_truths, 
            lang="en",
            model_type="microsoft/deberta-xlarge-mnli"
        )
        metrics['bert_score_p'] = sum(bert_results['precision']) / len(bert_results['precision'])
        metrics['bert_score_r'] = sum(bert_results['recall']) / len(bert_results['recall'])
        metrics['bert_score_f1'] = sum(bert_results['f1']) / len(bert_results['f1'])
    except Exception as e:
        print(f"BERTScore calculation error: {e}")
        metrics['bert_score_p'] = 0
        metrics['bert_score_r'] = 0
        metrics['bert_score_f1'] = 0
    
    # BARTScore calculation
    bart_scores = []
    try:
        with torch.no_grad():
            for pred, ref in zip(valid_predictions, valid_ground_truths):
                # Skip empty strings
                if not pred.strip() or not ref.strip():
                    continue
                
                # Encode the prediction and reference
                inputs = bart_tokenizer(ref, return_tensors="pt", max_length=1024, truncation=True).to(device)
                with torch.no_grad():
                    pred_encoded = bart_tokenizer(pred, return_tensors="pt", max_length=1024, truncation=True).to(device)
                    ref_ids = inputs["input_ids"]
                    
                    # Calculate log likelihood
                    outputs = bart_model(
                        input_ids=pred_encoded["input_ids"],
                        attention_mask=pred_encoded["attention_mask"],
                        labels=ref_ids
                    )
                    neg_log_likelihood = outputs.loss.item()
                    # Convert negative log likelihood to score (higher is better)
                    bart_score = -neg_log_likelihood
                    bart_scores.append(bart_score)
        
        metrics['bart_score'] = sum(bart_scores) / len(bart_scores) if bart_scores else 0
    except Exception as e:
        print(f"BARTScore calculation error: {e}")
        metrics['bart_score'] = 0
    
    # ROUGE-L Score calculation
    try:
        # Initialize rouge scorer with rouge-l
        scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        
        rouge_l_scores = []
        for pred, ref in zip(valid_predictions, valid_ground_truths):
            # Calculate ROUGE-L score
            scores = scorer.score(ref, pred)
            rouge_l_scores.append(scores['rougeL'].fmeasure)
        
        metrics['rouge_l'] = sum(rouge_l_scores) / len(rouge_l_scores) if rouge_l_scores else 0
    except Exception as e:
        print(f"ROUGE-L calculation error: {e}")
        metrics['rouge_l'] = 0
    
    return metrics

# Step 10: Define evaluation function with progress bar (optimized for single-word answers)
def evaluate_dataset(dataset_sample, name):
    print(f"\nEvaluating {name} dataset ({len(dataset_sample)} samples)...")
    predictions = []
    ground_truths = []
    
    # Track error cases to help with debugging
    missing_files = 0
    other_errors = 0
    
    # Also track error cases by type
    error_types = Counter()
    
    # Use tqdm for progress bar
    for example in tqdm(dataset_sample, desc=f"Processing {name} dataset"):
        try:
            # Use the optimized prediction function with unique answers from dataset
            predicted_answer = get_prediction(example, unique_answers)
            ground_truth = example['answer']
            
            # Skip if ground_truth or predicted_answer is invalid
            if (ground_truth is None or not isinstance(ground_truth, str) or
                predicted_answer is None or not isinstance(predicted_answer, str)):
                continue
            
            # For single-word answers, normalize by trimming and lowercasing
            predictions.append(predicted_answer.lower().strip())
            ground_truths.append(ground_truth.lower().strip())
        
        except FileNotFoundError as e:
            missing_files += 1
            if missing_files < 10:  # Only print first few errors to avoid spam
                print(f"FileNotFoundError: {e}")
            continue
        except Exception as e:
            other_errors += 1
            error_type = type(e).__name__
            error_types[error_type] += 1
            if other_errors < 10:  # Only print first few errors to avoid spam
                print(f"Error processing example: {e}")
            continue
    
    # Report error statistics
    if missing_files > 0:
        print(f"Missing files: {missing_files}")
    if other_errors > 0:
        print(f"Other errors: {other_errors}")
        print("Error types distribution:", dict(error_types))
    
    # Calculate metrics
    metrics = calculate_metrics(predictions, ground_truths)
    
    # Print metrics with focus on accuracy for single-word answers
    print(f"\n{name} Evaluation Results:")
    print(f"Number of samples evaluated: {len(predictions)}")
    print(f"Exact Match Accuracy: {metrics['accuracy'] * 100:.2f}%")
    print(f"Close Match Accuracy (with typo tolerance): {metrics['close_match_accuracy'] * 100:.2f}%")
    
    # Print other metrics
    if 'f1_score' in metrics:
        print(f"F1 Score: {metrics['f1_score'] * 100:.2f}%")
    if 'bleu_score' in metrics:
        print(f"BLEU Score: {metrics['bleu_score'] * 100:.2f}%")
    if 'bert_score_f1' in metrics:
        print(f"BERTScore F1: {metrics['bert_score_f1'] * 100:.2f}%")
    if 'rouge_l' in metrics:
        print(f"ROUGE-L Score: {metrics['rouge_l'] * 100:.2f}%")
    
    # For single-word answers, create confusion matrix for most common errors
    # This is extremely helpful for debugging and improving performance
    errors = [(pred, truth) for pred, truth in zip(predictions, ground_truths) if pred != truth]
    if errors:
        print("\nTop misclassifications:")
        error_pairs = Counter(errors)
        for (pred, truth), count in error_pairs.most_common(10):
            print(f"Predicted '{pred}' instead of '{truth}': {count} times")
    
    return metrics

# Step 11: Create different sized datasets
print(f"\nTotal dataset size: {len(full_dataset['train'])} samples")

# Create different sized datasets
full_sample = full_dataset['train']
half_sample = create_sampled_dataset(full_dataset, 0.5)
quarter_sample = create_sampled_dataset(full_dataset, 0.25)

print(f"Full dataset: {len(full_sample)} samples")
print(f"50% dataset: {len(half_sample)} samples")
print(f"25% dataset: {len(quarter_sample)} samples")

# Step 12: Evaluate and collect results
results = {}

# Evaluate quarter dataset
results['25% Dataset'] = evaluate_dataset(quarter_sample, "25% Dataset")

# Evaluate half dataset
results['50% Dataset'] = evaluate_dataset(half_sample, "50% Dataset")

# Evaluate full dataset
results['Full Dataset'] = evaluate_dataset(full_sample, "Full Dataset")

# Step 13: Create comparison charts (optimized for single-word answers)
def plot_metrics_comparison(results):
    # For single-word answers, focus on accuracy metrics
    metrics_to_plot = ['accuracy', 'close_match_accuracy']
    if 'f1_score' in results['Full Dataset']:
        metrics_to_plot.extend(['f1_score', 'bleu_score', 'bert_score_f1', 'rouge_l'])
    
    display_names = {
        'accuracy': 'Exact Match Accuracy',
        'close_match_accuracy': 'Close Match Accuracy',
        'f1_score': 'F1 Score',
        'bleu_score': 'BLEU Score',
        'bert_score_f1': 'BERTScore F1',
        'rouge_l': 'ROUGE-L'
    }
    
    # Create figure with subplots - 2 rows of 3 metrics each
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Plot each metric
    axes = axes.flatten()
    
    for i, metric in enumerate(metrics_to_plot):
        dataset_names = list(results.keys())
        
        # Handle BARTScore differently as it's not a percentage
        if metric == 'bart_score':
            values = [results[dataset][metric] for dataset in dataset_names]
            bars = axes[i].bar(dataset_names, values)
            axes[i].set_title(display_names[metric])
            axes[i].set_ylabel('Score')
            
            # Add value labels on bars
            for bar in bars:
                height = bar.get_height()
                axes[i].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                             f'{height:.4f}', ha='center', va='bottom')
        else:
            values = [results[dataset][metric] * 100 for dataset in dataset_names]
            bars = axes[i].bar(dataset_names, values)
            axes[i].set_title(display_names[metric])
            axes[i].set_ylabel('Score (%)')
            axes[i].set_ylim(0, 100)
            
            # Add value labels on bars
            for bar in bars:
                height = bar.get_height()
                axes[i].text(bar.get_x() + bar.get_width()/2., height + 1,
                             f'{height:.2f}%', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig('clip_metrics_comparison.png')
    plt.close()
    
    # Create dataset size comparison
    dataset_sizes = {
        '25% Dataset': len(quarter_sample),
        '50% Dataset': len(half_sample),
        'Full Dataset': len(full_sample)
    }
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(dataset_sizes.keys(), dataset_sizes.values())
    plt.title('Dataset Size Comparison')
    plt.ylabel('Number of Samples')
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                 f'{height}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig('dataset_size_comparison.png')
    plt.close()

# Step 14: Save results to CSV (optimized for single-word answers)
results_dict = {
    'Dataset': list(results.keys()),
    'Size': [len(quarter_sample), len(half_sample), len(full_sample)],
    'Exact Match Accuracy (%)': [results[ds]['accuracy'] * 100 for ds in results],
    'Close Match Accuracy (%)': [results[ds]['close_match_accuracy'] * 100 for ds in results],
}

# Add other metrics if they exist
if 'f1_score' in results['Full Dataset']:
    results_dict.update({
        'F1 Score (%)': [results[ds]['f1_score'] * 100 for ds in results if 'f1_score' in results[ds]],
        'BLEU Score (%)': [results[ds]['bleu_score'] * 100 for ds in results if 'bleu_score' in results[ds]],
        'BERTScore F1 (%)': [results[ds]['bert_score_f1'] * 100 for ds in results if 'bert_score_f1' in results[ds]],
        'ROUGE-L (%)': [results[ds]['rouge_l'] * 100 for ds in results if 'rouge_l' in results[ds]]
    })

results_df = pd.DataFrame(results_dict)

# Add information about unique answers and approach
print(f"\nUsing {len(unique_answers)} unique single-word answers from the dataset")

results_df.to_csv('clip_evaluation_results.csv', index=False)
print("\nResults saved to clip_evaluation_results.csv")
print("\nEvaluation complete!")

# Additional analysis for single-word answers
print("\nAnalysis of single-word answers in the dataset:")
all_answers = [example['answer'].lower().strip() for example in full_dataset['train'] 
              if isinstance(example['answer'], str) and example['answer'].strip()]
answer_counts = Counter(all_answers)

# Print the top answers with their frequencies
print("\nTop 30 most common answers in the dataset:")
for answer, count in answer_counts.most_common(30):
    print(f"{answer}: {count} occurrences")

# Print distribution statistics
total_answers = len(all_answers)
unique_count = len(answer_counts)
top_20_count = sum(count for _, count in answer_counts.most_common(20))
top_100_count = sum(count for _, count in answer_counts.most_common(100))

print(f"\nTotal answers in dataset: {total_answers}")
print(f"Unique answers: {unique_count}")
print(f"Top 20 answers cover {top_20_count/total_answers*100:.2f}% of all examples")
print(f"Top 100 answers cover {top_100_count/total_answers*100:.2f}% of all examples")

# Create a histogram of answer frequencies
plt.figure(figsize=(12, 6))
frequency_counts = Counter([count for _, count in answer_counts.items()])
items = sorted(frequency_counts.items())
x, y = zip(*items)
plt.bar(range(len(x)), y, align='center')
plt.xticks(range(len(x)), x, rotation=90)
plt.xlabel('Answer Frequency')
plt.ylabel('Number of Unique Answers')
plt.title('Distribution of Answer Frequencies')
plt.tight_layout()
plt.savefig('answer_frequency_distribution.png')
plt.close()

print("\nSaved answer frequency distribution to 'answer_frequency_distribution.png'")

# Analyze answer characteristics for single words
word_lengths = [len(answer) for answer in answer_counts.keys()]
avg_length = sum(word_lengths) / len(word_lengths)

print(f"\nAverage answer length: {avg_length:.2f} characters")
print(f"Shortest answer: {min(answer_counts.keys(), key=len)} ({len(min(answer_counts.keys(), key=len))} chars)")
print(f"Longest answer: {max(answer_counts.keys(), key=len)} ({len(max(answer_counts.keys(), key=len))} chars)")

# Save list of all unique answers for reference
with open('unique_answers.txt', 'w') as f:
    for answer in unique_answers:
        f.write(f"{answer}\n")

print("\nSaved list of unique answers used to 'unique_answers.txt'")

Input directory: ['imagedataset', 'dataset']
vqa-dataset contents: ['Complete_vqa(in).csv']
small folder contents: ['d8', '0d', '47', '17', '81', 'c5', '19', 'e2', '8f', 'a2', '22', 'cf', 'b8', '35', '92', 'b2', '50', 'c7', '23', '5b', '87', '07', 'd5', '10', 'd0', 'cd', '61', '1b', 'bf', '2e', '36', '05', '20', '06', '0f', '45', '60', '27', '6d', '64', '41', 'c8', '9e', 'da', 'ff', '89', '39', '0c', '4d', '02', '32', '98', '25', '42', 'b1', 'e5', 'b4', '52', 'f0', 'ac', 'd9', 'e0', '75', '0a', '8b', '2c', '38', '7d', '12', 'dc', 'ef', '94', '55', 'a0', 'e1', 'e7', '04', 'be', 'f1', '7e', 'ea', '49', 'fc', 'e3', 'e4', 'b9', '6e', '31', 'f9', '62', 'b6', '53', '1a', 'd2', '70', '34', '18', '4b', 'db', '79', '85', 'c2', '88', '65', '1e', '67', 'ec', 'ab', 'a3', 'a7', '78', '0e', 'd6', '28', 'f4', '66', 'cb', 'a5', 'images.csv', 'a8', 'bc', 'e6', '56', '72', '6c', '16', '7b', 'bb', 'af', '9f', 'bd', 'ba', '13', '99', '1d', '4e', '3e', '7f', '26', 'b5', 'fb', 'd4', '4c', '74', 'b0', 'c6', 

Processing 25% Dataset dataset: 100%|██████████| 11370/11370 [4:18:09<00:00,  1.36s/it] 


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.04G [00:00<?, ?B/s]


25% Dataset Evaluation Results:
Number of samples evaluated: 11360
Exact Match Accuracy: 4.79%
Close Match Accuracy (with typo tolerance): 5.33%
F1 Score: 4.79%
BLEU Score: 0.86%
BERTScore F1: 51.15%
ROUGE-L Score: 5.21%

Top misclassifications:
Predicted 'phone' instead of 'yes': 193 times
Predicted 'samsung' instead of 'yes': 157 times
Predicted 'phone' instead of 'black': 123 times
Predicted 'phone' instead of 'pink': 93 times
Predicted 'phone' instead of 'blue': 84 times
Predicted 'phone' instead of 'white': 58 times
Predicted 'motorola' instead of 'yes': 56 times
Predicted 'amazonbasics' instead of 'yes': 54 times
Predicted 'phone' instead of 'yellow': 53 times
Predicted 'samsung' instead of 'blue': 53 times

Evaluating 50% Dataset dataset (22741 samples)...


Processing 50% Dataset dataset:  17%|█▋        | 3963/22741 [1:30:33<7:09:03,  1.37s/it]


KeyboardInterrupt: 